In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data

In [ ]:
!pip install yfinance
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=5318f79ea56502557393c23440c06351716523b42deba2447aa6e483865a357c
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
import tqdm
import yfinance as yf
from torch.utils.data import DataLoader
from torch.distributions import Categorical
import tqdm
from google.colab import runtime
# Finance Data
import pandas as pd
import pandas_ta as ta
from typing import List

import numpy as np
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset
from torch.utils.data import DataLoader


seaborn.set_context(context="talk")
%matplotlib inline

In [ ]:
class TimeFeature:
    def __init__(self):
        pass

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        pass

    def __repr__(self):
        return self.__class__.__name__ + "()"


class SecondOfMinute(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.second / 59.0 - 0.5


class MinuteOfHour(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.minute / 59.0 - 0.5


class HourOfDay(TimeFeature):
    """Hour of day encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.hour / 23.0 - 0.5


class DayOfWeek(TimeFeature):
    """Day of week encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.dayofweek / 6.0 - 0.5


class DayOfMonth(TimeFeature):
    """Day of month encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.day - 1) / 30.0 - 0.5


class DayOfYear(TimeFeature):
    """Day of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.dayofyear - 1) / 365.0 - 0.5


class MonthOfYear(TimeFeature):
    """Month of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.month - 1) / 11.0 - 0.5


class WeekOfYear(TimeFeature):
    """Week of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.isocalendar().week - 1) / 52.0 - 0.5


def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
    """
    Returns a list of time features that will be appropriate for the given frequency string.
    Parameters
    ----------
    freq_str
        Frequency string of the form [multiple][granularity] such as "12H", "5min", "1D" etc.
    """
    #print(f"freq_str: {freq_str}")

    features_by_offsets = {
        offsets.YearEnd: [],
        offsets.QuarterEnd: [MonthOfYear],
        offsets.MonthEnd: [MonthOfYear],
        offsets.Week: [DayOfMonth, WeekOfYear],
        offsets.Day: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.BusinessDay: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Hour: [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Minute: [
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
        offsets.Second: [
            SecondOfMinute,
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
    }

    offset = to_offset(freq_str)
    #print(offset)

    for offset_type, feature_classes in features_by_offsets.items():
        #print(f"offset_type: {offset_type}")
        if isinstance(offset, offset_type):
            #print(cls for cls in feature_classes)
            return [cls() for cls in feature_classes]

    supported_freq_msg = f"""
    Unsupported frequency {freq_str}
    The following frequencies are supported:
        Y   - yearly
            alias: A
        M   - monthly
        W   - weekly
        D   - daily
        B   - business days
        H   - hourly
        T   - minutely
            alias: min
        S   - secondly
    """
    raise RuntimeError(supported_freq_msg)


def time_features(dates, freq='h'):
    #print(f"Entered time_features!!")
    #print(dates)
    return np.vstack([feat(dates) for feat in time_features_from_frequency_str(freq)])


In [ ]:
def extract_data(start_year, end_year=2023, ticker="^SPX"):
    data = yf.Ticker(ticker).history(period="max")
    data = data.dropna()

    def add_features(data):
      # Assuming your DataFrame is named 'data'
      data.ta.sma(close="Close", length=50, append=True)
      data.ta.sma(close="Close", length=200, append=True)
      #data.ta.ichimoku(close="Close", append=True)
      #data.ta.macd(close="Close", append=True)
      data.ta.rsi(close="Close", append=True)
      data.ta.bbands(close="Close", append=True)
      data.ta.macd(close="Close", append=True)
      data.ta.ichimoku(close="Close", append=True)
      data.ta.smi(close="Close", append=True)
      data.ta.willr(close="Close", low="Low", high="High", append=True)
      data.ta.stoch(close="Close", low="Low", high="High", append=True)
      data.ta.fisher(low="Low", high="High", append=True)
      data.ta.atr(low="Low", high="High", close="Close", append=True)
      #data.ta.cdl_pattern(name=['eveningstar', '3whitesoldiers', 'morningstar', '3blackcrows', '3linestrike'])
      data.ta.obv(volume="Volume", close="Close", append=True)
      data.ta.zscore(close="Close", append=True)
      data.ta.entropy(close="Close", append=True)
      return data

    data = add_features(data)
    ## Columns to Drop
    #drop = ['Volume', 'Dividends', 'Stock Splits']
    #data = data.drop(drop, axis=1)
    data = data.dropna()
    start_year = data.index[0].year if start_year is None else start_year
    data = data[data.index.year >= start_year-1] if start_year is not None else data
    data = data[data.index.year <= end_year] if end_year is not None else data
    #print(f"start year: {start_year}")
    #print(data)
    return data

df = extract_data(1990)
#df = df['Close']
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_50,SMA_200,RSI_14,...,SMIo_5_20_5,WILLR_14,STOCHk_14_3_3,STOCHd_14_3_3,FISHERT_9_1,FISHERTs_9_1,ATRr_14,OBV,ZS_30,ENTP_10
Date,,,,,,,,,,,,,,,,,,,,,
1989-01-03 00:00:00-05:00,277.720001,277.720001,273.809998,275.309998,128500000,0.0,0.0,274.762200,268.111900,48.838065,...,-0.057734,-77.409688,54.631409,58.669202,0.388937,0.797978,2.233492,2.680319e+10,0.229301,3.325722
1989-01-04 00:00:00-05:00,275.309998,279.750000,275.309998,279.429993,149700000,0.0,0.0,274.677599,268.165350,58.288271,...,-0.011009,-15.361706,54.945716,57.522858,0.248036,0.388937,2.391100,2.695289e+10,1.197195,3.325502
1989-01-05 00:00:00-05:00,279.429993,281.510010,279.429993,280.010010,174040000,0.0,0.0,274.632200,268.221200,59.424554,...,0.014795,-19.480489,62.582706,57.386610,0.507648,0.248036,2.368880,2.712693e+10,1.312146,3.326466
1989-01-06 00:00:00-05:00,280.010010,282.059998,280.010010,280.670013,161330000,0.0,0.0,274.598000,268.280000,60.735306,...,0.030888,-16.848292,82.769838,66.766086,0.908969,0.507648,2.346102,2.728826e+10,1.453896,3.327660
1989-01-09 00:00:00-05:00,280.670013,281.890015,280.320007,280.980011,163180000,0.0,0.0,274.590000,268.368150,61.366579,...,0.037330,-13.090746,83.526824,76.293122,1.352505,0.908969,2.290666,2.745144e+10,1.477710,3.329150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,4753.919922,4772.939941,4736.770020,4754.629883,3046770000,0.0,0.0,4468.218584,4335.861046,71.069402,...,-0.054860,-10.098875,80.663261,83.908053,2.832599,2.986470,41.184150,1.379242e+12,1.638294,3.350524
2023-12-26 00:00:00-05:00,4758.859863,4784.720215,4758.450195,4774.750000,2513910000,0.0,0.0,4477.157988,4340.426846,72.704374,...,-0.035086,-4.185293,90.737754,85.128169,2.967104,2.832599,40.391734,1.381756e+12,1.736890,3.350531
2023-12-27 00:00:00-05:00,4773.450195,4785.390137,4768.899902,4781.580078,2748450000,0.0,0.0,4485.316992,4345.055946,73.256889,...,-0.019366,-1.730508,94.661774,88.687597,3.241961,2.967104,38.684484,1.384504e+12,1.720615,3.349759


In [ ]:
import pandas as pd
import numpy as np

def datetime_to_one_hot(datetime_values):
    # Convert input to a DatetimeIndex if it's not already
    if isinstance(datetime_values, pd.Series):
        datetime_values = pd.to_datetime(datetime_values)
    elif isinstance(datetime_values, pd.Index):
        datetime_values = pd.DatetimeIndex(datetime_values)
    else:
        raise ValueError("Input should be a Pandas Series or DatetimeIndex.")

    # Extract the day of the week (0=Monday, 6=Sunday)
    day_of_week = datetime_values.dayofweek

    # Extract the day of the month (1-31)
    day_of_month = datetime_values.day

    # Extract the week of the month (1-5)
    week_of_month = (datetime_values.day - 1) // 7 + 1

    # Extract the month of the year (1-12)
    month_of_year = datetime_values.month

    # One-hot encode each component
    one_hot_day_of_week = np.eye(7)[day_of_week]
    one_hot_day_of_month = np.eye(31)[day_of_month - 1]  # day_of_month ranges from 1-31, so subtract 1 for 0-based indexing
    one_hot_week_of_month = np.eye(5)[week_of_month - 1]  # week_of_month ranges from 1-5, so subtract 1 for 0-based indexing
    one_hot_month_of_year = np.eye(12)[month_of_year - 1]  # month_of_year ranges from 1-12, so subtract 1 for 0-based indexing

    # Concatenate all one-hot vectors along the second axis
    one_hot_combined = np.concatenate([one_hot_day_of_week, one_hot_day_of_month, one_hot_week_of_month, one_hot_month_of_year], axis=1)

    return one_hot_combined


In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

class StockDataset(Dataset):
    def __init__(self, tickers, flag='train', size=None, one_hot_datetime=False,
                 features='S', target='Close', scale=True, timeenc=0, freq='d', batch_size=5,
                 data_start_year=1990, data_end_year=2023):
        if size is None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]

        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.data_start_year = data_start_year
        self.data_end_year = data_end_year

        self.features = features
        self.target = target
        self.scale = scale
        self.one_hot_datetime = one_hot_datetime
        self.timeenc = timeenc
        self.freq = freq
        self.scaler = StandardScaler()
        self.tickers = tickers.split()
        self.ticker_database = {}
        self.batch_size = batch_size
        self.min_year = 0
        self.__read_data__()

    def __len__(self):
        return len(self.years) - 1

    def get_min_year(self):
        min_year = 0
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_data(start_year=None, ticker=ticker)
            min_year = max(min_year, self.ticker_database[ticker].index.year.min())
        return min_year + 2

    def __read_data__(self):
        print(f"Loading following tickers: {self.tickers}\n")

        self.min_year = max(self.get_min_year(), self.data_start_year)
        print(f"Dataset Start Year: {self.min_year} | End Year: {self.data_end_year}")
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_data(start_year=self.min_year, end_year=self.data_end_year, ticker=ticker)
            self.ticker_database[ticker]['year'] = self.ticker_database[ticker].index.year

        self.years = self.ticker_database[self.tickers[0]]['year'].unique()
        print(f"years: {self.years}")
        self.data_by_year = {year: {ticker: self.ticker_database[ticker][self.ticker_database[ticker]['year'] == year] for ticker in self.tickers} for year in self.years}

        self.data_len = len(self.years)
        print(f"DateTime is one-hot: {self.one_hot_datetime}")

    def __getitem__(self, idx):
        """
        Final item form is a list containing [seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates]
        batches_x = batches[0]
        batches_y = batches[1]
        batches_x_mark = batches[2]
        batches_y_mark = batches[3]
        batches_x_dates = batches[4]
        batches_y_dates = batches[5]

        Each batches, contain batch data of size (batch_size, seq_len, num_features)
        For example, if seq_len = 24, batch_size = 5, num_features = 32,
        Each item of batches_x is a tensor of (5, 24, 32)
        """
        year = self.min_year + idx if idx >= 0 else self.max_year + idx + 1
        #print(f"Stock Dataset Yeaer: {year}")
        raw_datas = []

        for ticker in self.tickers:
            if year - 1 in self.data_by_year:
                prev_year_data = self.data_by_year[year - 1][ticker].tail(self.seq_len + self.pred_len - 1)
                #print(f"Prev year data for {ticker} in {year}: {prev_year_data.index}")
            else:
                print(f"Previous Year Data is Insufficient, Year: {year-1}, Ticker: {ticker}")
                prev_year_data = pd.DataFrame()

            ticker_data = pd.concat([prev_year_data, self.data_by_year[year][ticker]])
            #print(f"Ticker data for {ticker} in {year}: {ticker_data.index}")
            ticker_data['date'] = ticker_data.index
            raw_datas.append(ticker_data)
            """print(f"Ticker: {ticker}")
            print(len(ticker_data))
            print(ticker_data)"""

        seq_x = []
        seq_y = []
        seq_x_mark = []
        seq_y_mark = []
        seq_x_dates = []
        seq_y_dates = []

        for item in raw_datas:
            x, y, x_mark, y_mark, x_dates, y_dates = self.make_data(item)
            seq_x.append(x)
            seq_y.append(y)
            seq_x_mark.append(x_mark)
            seq_y_mark.append(y_mark)
            seq_x_dates.append(x_dates)
            seq_y_dates.append(y_dates)

        # Combine all tickers into a single batch and slice them into mini-batches
        #print(len(seq_x))
        return self.create_batches(seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates), year

    def make_data(self, raw_data):
        cols = list(raw_data.columns)
        cols.remove(self.target)
        cols.remove('date')
        cols.remove('year')
        raw_data = raw_data[['date'] + cols + [self.target]]

        if self.features == 'MS':
            cols_data = raw_data.columns[1:]
            data = raw_data[cols_data]
        elif self.features == 'S':
            data = raw_data['Close']

        if self.scale:
            data = self.scaler.fit_transform(data.values)

        if self.one_hot_datetime:
          #print(type(pd.to_datetime(raw_data['date'].values)))
          data_stamp = datetime_to_one_hot(pd.to_datetime(raw_data['date'].values))
          #print(one_hot_data_stamp.shape)
        else:
          data_stamp = time_features(pd.to_datetime(raw_data['date'].values), freq=self.freq)
          data_stamp = data_stamp.transpose(1, 0)

        """print("----Yearly data----")
        print(data[0].shape)
        print(len(data))
        print(data)"""

        seq_x = []
        seq_y = []
        seq_x_mark = []
        seq_y_mark = []
        x_dates = []
        y_dates = []

        for i in range(len(data) - self.pred_len - self.seq_len + 1):
            s_begin = i
            s_end = s_begin + self.seq_len
            r_begin = s_end - self.label_len
            r_end = r_begin + self.label_len + self.pred_len

            seq_x.append(data[s_begin:s_end])
            seq_y.append(data[r_begin:r_end])
            seq_x_mark.append(data_stamp[s_begin:s_end])
            seq_y_mark.append(data_stamp[r_begin:r_end])
            x_dates.append(raw_data['date'][s_begin:s_end])
            y_dates.append(raw_data['date'][r_begin:r_end])
            """print(f"data[s_begin:s_end]: {data[s_begin:s_end]}")
            print(f"data[r_begin:r_end]: {data[r_begin:r_end]}")
            print(f"raw_data['date'][s_begin:s_end]: {raw_data['date'][s_begin:s_end]}")
            print(f"raw_data['date'][r_begin:r_end]: {raw_data['date'][r_begin:r_end]}")
            print("=======================================")"""

        return torch.tensor(np.array(seq_x)), torch.tensor(np.array(seq_y)), torch.tensor(np.array(seq_x_mark)), torch.tensor(np.array(seq_y_mark)), x_dates, y_dates

    def create_batches(self, seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates, dates=None):
        batches_x = []
        batches_y = []
        batches_x_mark = []
        batches_y_mark = []
        batches_x_dates = []
        batches_y_dates = []
        batch_size = self.batch_size
        #print(dates[-1])
        for x, y, x_mark, y_mark, x_dates, y_dates in zip(seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates):
            #print(x.shape)
            for i in range(0, x.shape[0], batch_size):
                batches_x.append(x[i:i + batch_size])
                batches_y.append(y[i:i + batch_size])
                #print(f"x[i:i + batch_size]: {x[i:i + batch_size].shape}")
                #print(f"y[i:i + batch_size]: {y[i:i + batch_size].shape}")
                batches_x_mark.append(x_mark[i:i + batch_size])
                batches_y_mark.append(y_mark[i:i + batch_size])
                batches_x_dates.append(x_dates[i:i + batch_size])
                batches_y_dates.append(y_dates[i:i + batch_size])
                """print(f"x[i:i + batch_size]: {x[i:i + batch_size]}")
                print(f"y[i:i + batch_size]: {y[i:i + batch_size]}")
                print(f"x_dates[i:i + batch_size]: {x_dates[i:i + batch_size]}")
                print(f"y_dates[i:i + batch_size]: {y_dates[i:i + batch_size]}")
                print(f"------------------------------------------------------")"""

        return batches_x, batches_y, batches_x_mark, batches_y_mark, batches_x_dates, batches_y_dates

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [ ]:
def collate_fn(batch):
    seq_x, seq_y, seq_x_mark, seq_y_mark, year = batch[0]
    return seq_x, seq_y, seq_x_mark, seq_y_mark, year

dataset = StockDataset(tickers='goog aapl msft',timeenc=1, freq='d', size=[36, 18, 1], features='MS', one_hot_datetime=True)

Loading following tickers: ['goog', 'aapl', 'msft']

Dataset Start Year: 2007 | End Year: 2023
years: [2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2020 2021 2022 2023]


In [ ]:
batches, _ = dataset[0]
batches[2][0].shape

torch.Size([5, 36, 55])

# Model

## Autoformer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class my_Layernorm(nn.Module):
    """
    Special designed layernorm for the seasonal part
    """

    def __init__(self, channels):
        super(my_Layernorm, self).__init__()
        self.layernorm = nn.LayerNorm(channels)

    def forward(self, x):
        x_hat = self.layernorm(x)
        bias = torch.mean(x_hat, dim=1).unsqueeze(1).repeat(1, x.shape[1], 1)
        return x_hat - bias


class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """

    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """

    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean


class series_decomp_multi(nn.Module):
    """
    Multiple Series decomposition block from FEDformer
    """

    def __init__(self, kernel_size):
        super(series_decomp_multi, self).__init__()
        self.kernel_size = kernel_size
        self.series_decomp = [series_decomp(kernel) for kernel in kernel_size]

    def forward(self, x):
        moving_mean = []
        res = []
        for func in self.series_decomp:
            sea, moving_avg = func(x)
            moving_mean.append(moving_avg)
            res.append(sea)

        sea = sum(res) / len(res)
        moving_mean = sum(moving_mean) / len(moving_mean)
        return sea, moving_mean


class EncoderLayer(nn.Module):
    """
    Autoformer encoder layer with the progressive decomposition architecture
    """

    def __init__(self, attention, d_model, d_ff=None, moving_avg=25, dropout=0.1, activation="relu"):
        super(EncoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.attention = attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1, bias=False)
        self.decomp1 = series_decomp(moving_avg)
        self.decomp2 = series_decomp(moving_avg)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, attn_mask=None):
        new_x, attn = self.attention(
            x, x, x,
            attn_mask=attn_mask
        )
        x = x + self.dropout(new_x)
        x, _ = self.decomp1(x)
        y = x
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))
        res, _ = self.decomp2(x + y)
        return res, attn


class Encoder(nn.Module):
    """
    Autoformer encoder
    """

    def __init__(self, attn_layers, conv_layers=None, norm_layer=None):
        super(Encoder, self).__init__()
        self.attn_layers = nn.ModuleList(attn_layers)
        self.conv_layers = nn.ModuleList(conv_layers) if conv_layers is not None else None
        self.norm = norm_layer

    def forward(self, x, attn_mask=None):
        attns = []
        if self.conv_layers is not None:
            for attn_layer, conv_layer in zip(self.attn_layers, self.conv_layers):
                x, attn = attn_layer(x, attn_mask=attn_mask)
                x = conv_layer(x)
                attns.append(attn)
            x, attn = self.attn_layers[-1](x)
            attns.append(attn)
        else:
            for attn_layer in self.attn_layers:
                x, attn = attn_layer(x, attn_mask=attn_mask)
                attns.append(attn)

        if self.norm is not None:
            x = self.norm(x)

        return x, attns


class DecoderLayer(nn.Module):
    """
    Autoformer decoder layer with the progressive decomposition architecture
    """

    def __init__(self, self_attention, cross_attention, d_model, c_out, d_ff=None,
                 moving_avg=25, dropout=0.1, activation="relu"):
        super(DecoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.self_attention = self_attention
        self.cross_attention = cross_attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1, bias=False)
        self.decomp1 = series_decomp(moving_avg)
        self.decomp2 = series_decomp(moving_avg)
        self.decomp3 = series_decomp(moving_avg)
        self.dropout = nn.Dropout(dropout)
        self.projection = nn.Conv1d(in_channels=d_model, out_channels=c_out, kernel_size=3, stride=1, padding=1,
                                    padding_mode='circular', bias=False)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, cross, x_mask=None, cross_mask=None):
        x = x + self.dropout(self.self_attention(
            x, x, x,
            attn_mask=x_mask
        )[0])
        x, trend1 = self.decomp1(x)
        x = x + self.dropout(self.cross_attention(
            x, cross, cross,
            attn_mask=cross_mask
        )[0])
        x, trend2 = self.decomp2(x)
        y = x
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))
        x, trend3 = self.decomp3(x + y)

        residual_trend = trend1 + trend2 + trend3
        residual_trend = self.projection(residual_trend.permute(0, 2, 1)).transpose(1, 2)
        return x, residual_trend


class Decoder(nn.Module):
    """
    Autoformer encoder
    """

    def __init__(self, layers, norm_layer=None, projection=None):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList(layers)
        self.norm = norm_layer
        self.projection = projection

    def forward(self, x, cross, x_mask=None, cross_mask=None, trend=None):
        for layer in self.layers:
            x, residual_trend = layer(x, cross, x_mask=x_mask, cross_mask=cross_mask)
            trend = trend + residual_trend

        if self.norm is not None:
            x = self.norm(x)

        if self.projection is not None:
            x = self.projection(x)
        return x, trend

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math
from math import sqrt
import os


class AutoCorrelation(nn.Module):
    """
    AutoCorrelation Mechanism with the following two phases:
    (1) period-based dependencies discovery
    (2) time delay aggregation
    This block can replace the self-attention family mechanism seamlessly.
    """

    def __init__(self, mask_flag=True, factor=1, scale=None, attention_dropout=0.1, output_attention=False):
        super(AutoCorrelation, self).__init__()
        self.factor = factor
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = nn.Dropout(attention_dropout)

    def time_delay_agg_training(self, values, corr):
        """
        SpeedUp version of Autocorrelation (a batch-normalization style design)
        This is for the training phase.
        """
        head = values.shape[1]
        channel = values.shape[2]
        length = values.shape[3]
        # find top k
        top_k = int(self.factor * math.log(length))
        mean_value = torch.mean(torch.mean(corr, dim=1), dim=1)
        index = torch.topk(torch.mean(mean_value, dim=0), top_k, dim=-1)[1]
        weights = torch.stack([mean_value[:, index[i]] for i in range(top_k)], dim=-1)
        # update corr
        tmp_corr = torch.softmax(weights, dim=-1)
        # aggregation
        tmp_values = values
        delays_agg = torch.zeros_like(values).float()
        for i in range(top_k):
            pattern = torch.roll(tmp_values, -int(index[i]), -1)
            delays_agg = delays_agg + pattern * \
                         (tmp_corr[:, i].unsqueeze(1).unsqueeze(1).unsqueeze(1).repeat(1, head, channel, length))
        return delays_agg

    def time_delay_agg_inference(self, values, corr):
        """
        SpeedUp version of Autocorrelation (a batch-normalization style design)
        This is for the inference phase.
        """
        batch = values.shape[0]
        head = values.shape[1]
        channel = values.shape[2]
        length = values.shape[3]
        # index init
        init_index = torch.arange(length).unsqueeze(0).unsqueeze(0).unsqueeze(0).repeat(batch, head, channel, 1).cuda()
        # find top k
        top_k = int(self.factor * math.log(length))
        mean_value = torch.mean(torch.mean(corr, dim=1), dim=1)
        weights, delay = torch.topk(mean_value, top_k, dim=-1)
        # update corr
        tmp_corr = torch.softmax(weights, dim=-1)
        # aggregation
        tmp_values = values.repeat(1, 1, 1, 2)
        delays_agg = torch.zeros_like(values).float()
        for i in range(top_k):
            tmp_delay = init_index + delay[:, i].unsqueeze(1).unsqueeze(1).unsqueeze(1).repeat(1, head, channel, length)
            pattern = torch.gather(tmp_values, dim=-1, index=tmp_delay)
            delays_agg = delays_agg + pattern * \
                         (tmp_corr[:, i].unsqueeze(1).unsqueeze(1).unsqueeze(1).repeat(1, head, channel, length))
        return delays_agg

    def time_delay_agg_full(self, values, corr):
        """
        Standard version of Autocorrelation
        """
        batch = values.shape[0]
        head = values.shape[1]
        channel = values.shape[2]
        length = values.shape[3]
        # index init
        init_index = torch.arange(length).unsqueeze(0).unsqueeze(0).unsqueeze(0).repeat(batch, head, channel, 1).cuda()
        # find top k
        top_k = int(self.factor * math.log(length))
        weights, delay = torch.topk(corr, top_k, dim=-1)
        # update corr
        tmp_corr = torch.softmax(weights, dim=-1)
        # aggregation
        tmp_values = values.repeat(1, 1, 1, 2)
        delays_agg = torch.zeros_like(values).float()
        for i in range(top_k):
            tmp_delay = init_index + delay[..., i].unsqueeze(-1)
            pattern = torch.gather(tmp_values, dim=-1, index=tmp_delay)
            delays_agg = delays_agg + pattern * (tmp_corr[..., i].unsqueeze(-1))
        return delays_agg

    def forward(self, queries, keys, values, attn_mask):
        B, L, H, E = queries.shape
        _, S, _, D = values.shape
        if L > S:
            zeros = torch.zeros_like(queries[:, :(L - S), :]).float()
            values = torch.cat([values, zeros], dim=1)
            keys = torch.cat([keys, zeros], dim=1)
        else:
            values = values[:, :L, :, :]
            keys = keys[:, :L, :, :]

        # period-based dependencies
        q_fft = torch.fft.rfft(queries.permute(0, 2, 3, 1).contiguous(), dim=-1)
        k_fft = torch.fft.rfft(keys.permute(0, 2, 3, 1).contiguous(), dim=-1)
        res = q_fft * torch.conj(k_fft)
        corr = torch.fft.irfft(res, dim=-1)

        # time delay agg
        if self.training:
            V = self.time_delay_agg_training(values.permute(0, 2, 3, 1).contiguous(), corr).permute(0, 3, 1, 2)
        else:
            V = self.time_delay_agg_inference(values.permute(0, 2, 3, 1).contiguous(), corr).permute(0, 3, 1, 2)

        if self.output_attention:
            return (V.contiguous(), corr.permute(0, 3, 1, 2))
        else:
            return (V.contiguous(), None)


class AutoCorrelationLayer(nn.Module):
    def __init__(self, correlation, d_model, n_heads, d_keys=None,
                 d_values=None):
        super(AutoCorrelationLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)
        d_values = d_values or (d_model // n_heads)

        self.inner_correlation = correlation
        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_model, d_keys * n_heads)
        self.value_projection = nn.Linear(d_model, d_values * n_heads)
        self.out_projection = nn.Linear(d_values * n_heads, d_model)
        self.n_heads = n_heads

    def forward(self, queries, keys, values, attn_mask):
        B, L, _ = queries.shape
        _, S, _ = keys.shape
        H = self.n_heads

        queries = self.query_projection(queries).view(B, L, H, -1)
        keys = self.key_projection(keys).view(B, S, H, -1)
        values = self.value_projection(values).view(B, S, H, -1)

        out, attn = self.inner_correlation(
            queries,
            keys,
            values,
            attn_mask
        )
        out = out.view(B, L, -1)

        return self.out_projection(out), attn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import math


class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()


class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()
        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(
            self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x


class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h': 4, 't': 5, 's': 6,
                    'm': 1, 'a': 1, 'w': 2, 'd': 3, 'b': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        return self.embed(x)


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x) + self.position_embedding(x)
        else:
            x = self.value_embedding(
                x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)


class DataEmbedding_wo_pos(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding_wo_pos, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        if x_mark is None:
            x = self.value_embedding(x)
        else:
            x = self.value_embedding(x) + self.temporal_embedding(x_mark)
        return self.dropout(x)


class PatchEmbedding(nn.Module):
    def __init__(self, d_model, patch_len, stride, padding, dropout):
        super(PatchEmbedding, self).__init__()
        # Patching
        self.patch_len = patch_len
        self.stride = stride
        self.padding_patch_layer = nn.ReplicationPad1d((0, padding))

        # Backbone, Input encoding: projection of feature vectors onto a d-dim vector space
        self.value_embedding = nn.Linear(patch_len, d_model, bias=False)

        # Positional embedding
        self.position_embedding = PositionalEmbedding(d_model)

        # Residual dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # do patching
        n_vars = x.shape[1]
        x = self.padding_patch_layer(x)
        x = x.unfold(dimension=-1, size=self.patch_len, step=self.stride)
        x = torch.reshape(x, (x.shape[0] * x.shape[1], x.shape[2], x.shape[3]))
        # Input encoding
        x = self.value_embedding(x) + self.position_embedding(x)
        return self.dropout(x), n_vars

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np


class Autoformer(nn.Module):
    """
    Autoformer is the first method to achieve the series-wise connection,
    with inherent O(LlogL) complexity
    Paper link: https://openreview.net/pdf?id=I55UqU-M11y
    """

    def __init__(self, configs):
        super(Autoformer, self).__init__()
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.label_len = configs.label_len
        self.pred_len = configs.pred_len
        self.output_attention = configs.output_attention

        # Decomp
        kernel_size = configs.moving_avg
        self.decomp = series_decomp(kernel_size)

        # Embedding
        self.enc_embedding = DataEmbedding_wo_pos(configs.enc_in, configs.d_model, configs.embed, configs.freq,
                                                  configs.dropout)
        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AutoCorrelationLayer(
                        AutoCorrelation(False, configs.factor, attention_dropout=configs.dropout,
                                        output_attention=configs.output_attention),
                        configs.d_model, configs.n_heads),
                    configs.d_model,
                    configs.d_ff,
                    moving_avg=configs.moving_avg,
                    dropout=configs.dropout,
                    activation=configs.activation
                ) for l in range(configs.e_layers)
            ],
            norm_layer=my_Layernorm(configs.d_model)
        )
        # Decoder
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            self.dec_embedding = DataEmbedding_wo_pos(configs.dec_in, configs.d_model, configs.embed, configs.freq,
                                                      configs.dropout)
            self.decoder = Decoder(
                [
                    DecoderLayer(
                        AutoCorrelationLayer(
                            AutoCorrelation(True, configs.factor, attention_dropout=configs.dropout,
                                            output_attention=False),
                            configs.d_model, configs.n_heads),
                        AutoCorrelationLayer(
                            AutoCorrelation(False, configs.factor, attention_dropout=configs.dropout,
                                            output_attention=False),
                            configs.d_model, configs.n_heads),
                        configs.d_model,
                        configs.c_out,
                        configs.d_ff,
                        moving_avg=configs.moving_avg,
                        dropout=configs.dropout,
                        activation=configs.activation,
                    )
                    for l in range(configs.d_layers)
                ],
                norm_layer=my_Layernorm(configs.d_model),
                projection=nn.Linear(configs.d_model, configs.c_out, bias=True)
            )
        if self.task_name == 'imputation':
            self.projection = nn.Linear(
                configs.d_model, configs.c_out, bias=True)
        if self.task_name == 'anomaly_detection':
            self.projection = nn.Linear(
                configs.d_model, configs.c_out, bias=True)
        if self.task_name == 'classification':
            self.datetime_linear = nn.Linear(configs.datetime_features, configs.d_model)
            self.act = F.gelu
            self.dropout = nn.Dropout(configs.dropout)
            self.projection = nn.Linear(
                configs.d_model * configs.seq_len, configs.num_class)

    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        # decomp init
        mean = torch.mean(x_enc, dim=1).unsqueeze(
            1).repeat(1, self.pred_len, 1)
        zeros = torch.zeros([x_dec.shape[0], self.pred_len,
                             x_dec.shape[2]], device=x_enc.device)
        seasonal_init, trend_init = self.decomp(x_enc)
        # decoder input
        trend_init = torch.cat(
            [trend_init[:, -self.label_len:, :], mean], dim=1)
        seasonal_init = torch.cat(
            [seasonal_init[:, -self.label_len:, :], zeros], dim=1)
        # enc
        enc_out = self.enc_embedding(x_enc, x_mark_enc)
        enc_out, attns = self.encoder(enc_out, attn_mask=None)
        # dec
        dec_out = self.dec_embedding(seasonal_init, x_mark_dec)
        seasonal_part, trend_part = self.decoder(dec_out, enc_out, x_mask=None, cross_mask=None,
                                                 trend=trend_init)
        # final
        dec_out = trend_part + seasonal_part
        return dec_out

    def classification(self, x_enc, x_mark_enc):
        # enc
        enc_out = self.enc_embedding(x_enc, None)
        enc_out, attns = self.encoder(enc_out, attn_mask=None)

        # Output
        # the output transformer encoder/decoder embeddings don't include non-linearity
        output = self.act(enc_out)
        output = self.dropout(output)
        # zero-out padding embeddings
        x_mark_enc = self.datetime_linear(x_mark_enc)
        #print(f"output shape: {output.shape} | x_mark_enc shape: {x_mark_enc.shape}")

        output = output * x_mark_enc
        # (batch_size, seq_length * d_model)
        output = output.reshape(output.shape[0], -1)
        output = self.projection(output)  # (batch_size, num_classes)
        return output

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            dec_out = self.forecast(x_enc, x_mark_enc, x_dec, x_mark_dec)
            return dec_out[:, -self.pred_len:, :]  # [B, L, D]
        if self.task_name == 'imputation':
            dec_out = self.imputation(
                x_enc, x_mark_enc, x_dec, x_mark_dec, mask)
            return dec_out  # [B, L, D]
        if self.task_name == 'anomaly_detection':
            dec_out = self.anomaly_detection(x_enc)
            return dec_out  # [B, L, D]
        if self.task_name == 'classification':
            #print(f"x_enc: {x_enc.shape} | x_mark_enc: {x_mark_enc.shape}")
            dec_out = self.classification(x_enc, x_mark_enc)
            return dec_out  # [B, N]
        return None

In [ ]:
import torch

def evaluate_direction_accuracy(output, batch_y):
    """
    Evaluate the accuracy of direction predictions and compare it to a baseline model.

    Parameters:
    output (torch.Tensor): Tensor of predicted values with shape (batch_size, 96, 1).
    batch_y (torch.Tensor): Tensor of true values with shape (batch_size, 96, 1).

    Returns:
    correct_predictions (int): Number of times the predicted direction was correct.
    accuracy (float): Fraction of correct predictions out of total comparisons.
    baseline_correct (int): Number of times the baseline prediction was correct.
    baseline_accuracy (float): Fraction of correct predictions by the baseline out of total comparisons.
    """
    # Compute the direction of change for each tensor
    def compute_direction(tensor):
        # Compute the difference between consecutive elements
        diff = tensor[:, 1:, :] - tensor[:, :-1, :]
        # Convert differences to binary direction indicators: 1 for increase, -1 for decrease
        direction = torch.sign(diff)
        return direction

    # Get direction indicators for both tensors
    #print(batch_y[:,0,:].shape)
    output = torch.cat((batch_y[:, 0, :].unsqueeze(2), output), dim=1)
    #print(output)
    output_direction = compute_direction(output)
    batch_y_direction = compute_direction(batch_y)
    #print(f"output_direction: {output_direction.shape} batch_y_direction: {batch_y_direction.shape}")
    # Compare the directions
    correct_predictions = (output_direction == batch_y_direction).sum().item()
    #print(f"correct_predictions: {correct_predictions}")
    total_comparisons = output_direction.numel()
    #print(f"total_comparisons: {total_comparisons}")
    accuracy = correct_predictions / total_comparisons

    # Compute baseline predictions (always predict increase)
    baseline_direction = torch.ones_like(output_direction)  # Baseline always predicts increase
    baseline_correct = (baseline_direction == batch_y_direction).sum().item()
    baseline_accuracy = baseline_correct / total_comparisons

    return correct_predictions, baseline_correct, total_comparisons

# Example usage (replace these with your actual tensors)
output = torch.randn(10, 1, 1)  # Example output tensor
batch_y = torch.randn(10, 2, 1)  # Example batch_y tensor

correct_predictions, baseline_correct, total_comparisons = evaluate_direction_accuracy(output, batch_y)

In [ ]:
def compute_direction(tensor):
  batch_size = tensor.size(0)
  comparison = tensor[:, 1, 0] > tensor[:, 0, 0]
  # Convert the comparison result to a one-hot vector
  one_hot = torch.zeros(batch_size, 2)
  one_hot[comparison, 0] = 1  # [1, 0] when the last value is greater
  one_hot[~comparison, 1] = 1  # [0, 1] when the first value is greater
  return one_hot

tensor = torch.randn(5, 2, 1)
print(tensor)
compute_direction(tensor)

tensor([[[-0.3382],
         [ 1.4185]],

        [[-2.2873],
         [-0.0740]],

        [[-0.5220],
         [ 0.7992]],

        [[ 0.3773],
         [-1.2975]],

        [[ 1.1841],
         [ 0.5156]]])


tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])

In [ ]:
def _predict(autoformer, batch_x, batch_y, batch_x_mark, batch_y_mark, device):
        # decoder input
        dec_inp = torch.zeros_like(batch_y[:, -configs.pred_len:, :]).float()
        #print(f"real dec_inp: {dec_inp.shape} and batch_y[:, :configs.label_len, :]: {batch_y[:, :configs.label_len, :].shape}")
        dec_inp = torch.cat([batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(device)
        #print(f"dec_inp shape: {dec_inp.shape}")
        # encoder - decoder

        def _run_model():
            outputs = autoformer(batch_x, batch_x_mark, dec_inp, batch_y_mark)
            if configs.output_attention:
                outputs = outputs[0]
            return outputs


        outputs = _run_model()

        f_dim = -1 if configs.features == 'MS' else 0
        #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape}")
        batch_y_direction = batch_y[:, -(configs.pred_len+1):, f_dim:].to(device)
        #outputs = outputs[:, -configs.pred_len:, f_dim:]
        batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(device)
        truth_direction = compute_direction(batch_y_direction).to(device)
        #print(f"outputs: {outputs.shape} | batch_Y: {batch_y.shape} | batch_y_direction {batch_y_direction.shape}")
        #assert torch.equal(pre_batch_y[:, -configs.pred_len:, f_dim:], batch_y[:, -configs.pred_len:, f_dim:])
        return outputs, batch_y, batch_y_direction, truth_direction

# Test Model

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    ##
    freq: str = 'd'
    task_name: str = 'classification'
    num_class: int = 2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 8
    factor: int = 1
    enc_in: int = 35
    dec_in: int = 35
    c_out: int = 1
    d_model: int = 512
    d_ff: int = 2048
    moving_avg: int = 25
    factor: int = 1
    distil: bool = True
    output_attention: bool = False
    patience: int = 3
    learning_rate: float = 0.0001
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.05
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = True
    datetime_features: int = 55

    ## Training
    run_name: str = "test"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "aapl" #"^SPX" #"goog amzn wmt xom brk-a lly ge lin pld aapl nee"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "autoformer_classification"

In [ ]:
configs = Args()
autoformer = Autoformer(configs)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_optim = torch.optim.Adam(autoformer.parameters(), lr=configs.learning_rate)
loss_fn = nn.MSELoss()
dataset = StockDataset(tickers='^SPX',timeenc=1, freq='d', size=[36, 18, 1], features='MS', one_hot_datetime=True)

Loading following tickers: ['^SPX']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
range_limit = configs.rolling_window+configs.validation_years+configs.test_years
loop_range = tqdm.tqdm(range(len(dataset)-range_limit))
autoformer = autoformer.to(device)
for iteration in loop_range:
  for window_iteration in range(configs.window_epoch):
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    starting_idx = iteration // 10
    for i in range((configs.rolling_window+configs.validation_years+configs.test_years)):
        validation, test = False, False
        if i == configs.rolling_window:
          validation = True
        elif i == (configs.rolling_window+1):
          test = True
        else:
          batches, year  = dataset[i+iteration]
          batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
          for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
            model_optim.zero_grad()
            batch_x = batch_x.float().to(device)
            true_batch_y = batch_y.float().to(device)
            batch_x_mark = batch_x_mark.float().to(device)
            batch_y_mark = batch_y_mark.float().to(device)
            outputs, batch_y, batch_y_direction, truth_direction = _predict(autoformer, batch_x, true_batch_y, batch_x_mark, batch_y_mark, device)
            print(f"outputs shape: {outputs.shape} | batch_y: {batch_y.shape}")

            loss = loss_fn(outputs, truth_direction)
            epoch_loss.append(loss.item())
            loss.backward()
            model_optim.step()

            #print(f"outputs shape: {outputs.shape} | true_batch_y[:,-97:,:]: {true_batch_y[:,-97:,-1:].shape}")
            correct_predictions, baseline_correct, total_comparisons = evaluate_direction_accuracy(outputs, true_batch_y[:,-(configs.pred_len+1):,-1:])
            total_hits += correct_predictions
            total_data +=total_comparisons
            total_ol += baseline_correct
        print(f"Year: {year} total_comparisons: {total_data} training_hit_ratio: {total_hits/total_data} | ol_hit_ratio: {total_ol/total_data}")
        break
    break
  break

  0%|          | 0/21 [00:00<?, ?it/s]

output shape: torch.Size([5, 36, 512]) | x_mark_enc shape: torch.Size([5, 36, 512])
outputs shape: torch.Size([5, 2]) | batch_y: torch.Size([5, 1, 1])


  0%|          | 0/21 [00:00<?, ?it/s]


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

# Rolling Trainer

In [ ]:
class Trainer():
  def __init__(self, configs):
    self.configs = configs

    ticker_str = configs.tickers.replace(" ", "_")
    self.run_name = f"[{ticker_str}]_valYrs:{self.configs.validation_years}_testYrs{self.configs.test_years}_reset:{self.configs.reset_model}_{configs.run_name}"
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.set_seed(configs.seed)
    self.dataset = self.make_data()
    self.model = Autoformer(configs)
    self.model.to(self.device)
    self.loss_fn = nn.BCEWithLogitsLoss()
    self.model_optim = torch.optim.Adam(self.model.parameters(), lr=configs.learning_rate)
    self.writer = SummaryWriter(f"/content/drive/MyDrive/code/fintransformer/runs/{self.run_name}")
    path = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}"
    save_folder = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}"
    if not os.path.exists(save_folder):
      os.mkdir(save_folder)
      print(f"Save Folder Made at {save_folder}")
    if not os.path.exists(path):
      os.mkdir(path)
      print(f"Save File Directory Made at {path}\n")
    else:
      print(f"Directory Already Exists at {path}")

  def set_seed(self, seed):
      torch.manual_seed(seed)
      torch.cuda.manual_seed(seed)
      #torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
      np.random.seed(seed)
      random.seed(seed)
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = False

  def make_data(self):
    def collate_fn(batch):
      seq_x, seq_y, seq_x_mark, seq_y_mark, year = batch[0]
      return seq_x, seq_y, seq_x_mark, seq_y_mark, year
    dataset = StockDataset(tickers=self.configs.tickers, timeenc=1, freq='d', size=[self.configs.seq_len, self.configs.label_len, self.configs.pred_len],
                           features=self.configs.features, batch_size=self.configs.batch_size, one_hot_datetime=self.configs.one_hot_datetime,
                           data_start_year=self.configs.data_start_year, data_end_year=self.configs.data_end_year)
    """prices, labels, year = next(iter(val_loader))
    feature_num = prices.shape[2]
    label_num = labels.shape[1]
    print(f"Data shape: {prices.shape} | Num Features: {feature_num}")"""
    return dataset


  def run(self):
    ### Training ###
    range_limit = self.configs.rolling_window+self.configs.validation_years+self.configs.test_years
    loop_range = tqdm.tqdm(range(len(self.dataset)-range_limit))
    recent_save_path = ""
    val_acc_arr = []
    test_acc_arr = []
    for iteration in loop_range:
      highest_acc = 0.0
      highest_test_acc = 0.0
      if self.configs.reset_model and iteration != 0:
        self.model = Autoformer(self.configs)
        self.model.to(self.device)
        self.model_optim = torch.optim.Adam(self.model.parameters(), lr=self.configs.learning_rate)
        print(f"Model Has Been Reset to Random Parameters")
      else:
        self.load_model(recent_save_path) if recent_save_path != "" else None
      for window_iteration in range(self.configs.window_epoch):
        train_loss, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year = self.train(iteration, window_iteration, highest_acc)
        if average_val_accuracy > highest_acc:
            highest_acc = average_val_accuracy
            highest_test_acc = average_test_accuracy
            recent_save_path = self.save_model(average_val_accuracy, average_test_accuracy, window_iteration, validation_year, test_year)
      #self.writer.add_scalar("Train/Test Loss", train_loss, iteration)
      val_acc_arr.append([highest_acc, validation_year])
      test_acc_arr.append([highest_test_acc, test_year])
      print(f"***********************************************************")
      print(f"Highest Validation Accuracy in {validation_year[0]}~{validation_year[-1]}: {highest_acc*100:.2f}% | Highest Test Accuracy in {test_year}: {average_test_accuracy*100:.2f}%")
      print(f"***********************************************************")
      print("=============================New Training Set====================================")
    self.writer.flush()
    self.writer.close()
    #print(f"Highest Accuracy in Validation Set: {highest_acc*100:.2f}")
    print()
    for val, test in zip(val_acc_arr, test_acc_arr):
      print(f"Validation Accuracy in {val[1][0]}~{val[1][-1]} : {val[0]*100:.2f}% | Test Accuracy in {test[1]}: {test[0]*100:.2f}%")

    average_first_elements_val = sum(item[0] for item in val_acc_arr) / len(val_acc_arr)
    average_first_elements_test = sum(item[0] for item in test_acc_arr) / len(test_acc_arr)
    print(f"Average Validation Accuracy: {average_first_elements_val*100:.2f}% | Average Test Accuracy: {average_first_elements_test*100:.2f}%")
    return

  def train(self, iteration, window_iteration, highest_acc):
    epoch_loss = []
    val_epoch_loss = []
    training_start_year, training_end_year = 0, 0
    training_total_hits = 0
    training_total_data = 0
    training_total_ol = 0
    validation_year, test_year = [], 0
    validation_total_hits = 0
    validation_total_data = 0
    validation_total_ol = 0
    test_total_hits = 0
    test_total_data = 0
    test_total_ol = 0
    for i in range((self.configs.rolling_window+self.configs.validation_years+self.configs.test_years)):
      validation, test = False, False
      if i >= self.configs.rolling_window and i <= (self.configs.rolling_window+self.configs.validation_years-1):
        batches, year = self.dataset[i+iteration]
        #print(f"validation year: {year}")
        #print(f"validation on year {year}: {batches[-1][0][0].iloc[0]} to {batches[-1][-1][-1].iloc[-1]}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        accuracy, only_long, val_loss_epoch, total_hits, total_data, total_ol = self.eval_once(batches_x, batches_y, batches_x_mark, batches_y_mark, year, highest_acc=0, train_acc=0)
        validation_total_hits += total_hits
        validation_total_data += total_data
        validation_total_ol += total_ol
        val_epoch_loss.append(val_loss_epoch)
        validation_year.append(year)
        validation = True
        #print(f"Year: {year} | Validation Accuracy: {(total_hits/total_data)*100:.2f}% | Only Long Accuracy: {(total_ol/total_data)*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
        self.model.train()
      elif i >= (self.configs.rolling_window+self.configs.validation_years):
        batches, year = self.dataset[i+iteration]
        #print(f"test year: {year}")
        #print(f"Test on year {year}: {batches[-1][0][0].iloc[0]} to {batches[-1][-1][-1].iloc[-1]}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        accuracy, only_long,  total_hits, total_data, total_ol = self.test(batches_x, batches_y, batches_x_mark, batches_y_mark, year)
        test_total_hits += total_hits
        test_total_data += total_data
        test_total_ol += total_ol
        test_year = year
        test = True
        self.model.train()
      else:
        batches, year = self.dataset[i+iteration]
        #print(f"train year: {year}")
        #print(batches[-1])
        #print(f"Train on year {year}: {batches[-1][0][0].iloc[0]} to {batches[-1][-1][-1].iloc[-1]}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        training_start_year = year if i == 0 else training_start_year
        training_end_year = year if i == self.configs.rolling_window-1 else training_end_year
        for j, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
          self.model_optim.zero_grad()
          batch_x = batch_x.float().to(self.device)
          batch_y = batch_y.float().to(self.device)
          batch_x_mark = batch_x_mark.float().to(self.device)
          batch_y_mark = batch_y_mark.float().to(self.device)
          #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
          outputs, true_batch_y, batch_y_direction, truth_direction = _predict(self.model, batch_x, batch_y, batch_x_mark, batch_y_mark, self.device)

          loss = self.loss_fn(outputs, truth_direction)
          epoch_loss.append(loss.item())
          loss.backward()
          self.model_optim.step()
        #print(f"outputs: {outputs.shape} | batch_y: {batch_y.shape}")
        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)

          p = outputs.detach().cpu().numpy().argmax(axis=1)
          l = truth_direction.detach().cpu().numpy().argmax(axis=1)
          only_long = np.zeros_like(l)
          correct_preds = np.sum(p == l)
          only_longs = np.sum(only_long == l)
          #print(f"outputs: {outputs.shape} | batch_y: {batch_y.shape}")
          #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)

          training_total_hits += correct_preds
          training_total_data += outputs.size(0)
          training_total_ol += only_longs

    loss_epoch = np.mean(epoch_loss)
    average_val_accuracy = validation_total_hits / validation_total_data
    average_val_only_long = validation_total_ol / validation_total_data
    average_val_loss = np.mean(val_epoch_loss)
    average_test_accuracy = test_total_hits / test_total_data
    average_test_only_long = test_total_ol / test_total_data
    train_accuracy = training_total_hits / training_total_data
    train_only_long = training_total_ol / training_total_data
    print(f"Training Years: {training_start_year}~{training_end_year} | Training Accuracy: {train_accuracy*100:.2f}% | Training OL Accuracy: {train_only_long*100:.2f}%")
    print(f"Year: {validation_year[0]}~{validation_year[-1]} Validation Accuracy: {average_val_accuracy*100:.2f}% | Only Long Accuracy: {average_val_only_long*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
    print(f"Year: {test_year} | Test Accuracy: {average_test_accuracy*100:.2f}% | Only Long Accuracy: {average_test_only_long*100:.2f}%")
    print(f"------------------------------------------------")
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Accuracy", train_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Loss", loss_epoch, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Loss", average_val_loss, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Accuracy", average_val_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Test Accuracy", average_test_accuracy, window_iteration)
      #print(f"Year: {year} training_hit_ratio: {hit_count/batch_x.shape[0]} | ol_hit_ratio: {only_long/batch_x.shape[0]}")

    #print(f"train accuracy: {accuracy*100:.2f}% | only_long: {only_long*100:.2f}%")
    return loss_epoch, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year

  def eval_once(self, batches_x, batches_y, batches_x_mark, batches_y_mark, year, highest_acc, train_acc, last=False):
    self.model.eval()
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)
        outputs, true_batch_y, batch_y_direction, truth_direction = _predict(self.model, batch_x, batch_y, batch_x_mark, batch_y_mark, self.device)

        loss = self.loss_fn(outputs, truth_direction)
        epoch_loss.append(loss.item())

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        p = outputs.detach().cpu().numpy().argmax(axis=1)
        l = truth_direction.detach().cpu().numpy().argmax(axis=1)
        only_long = np.zeros_like(l)
        correct_preds = np.sum(p == l)
        only_longs = np.sum(only_long == l)
        #print(f"outputs: {outputs.shape} | batch_y: {batch_y.shape}")
        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)

        total_hits += correct_preds
        total_data += outputs.size(0)
        total_ol += only_longs
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    loss_epoch = np.mean(epoch_loss)
    #print(f"validation accuracy: {accuracy*100:.2f}% | val highest_acc: {highest_acc*100:.2f}% | val only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, loss_epoch, total_hits, total_data, total_ol

  def test(self, batches_x, batches_y, batches_x_mark, batches_y_mark, year):
    self.model.eval()
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)
        outputs, true_batch_y, batch_y_direction, truth_direction = _predict(self.model, batch_x, batch_y, batch_x_mark, batch_y_mark, self.device)

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        p = outputs.detach().cpu().numpy().argmax(axis=1)
        l = truth_direction.detach().cpu().numpy().argmax(axis=1)
        only_long = np.zeros_like(l)
        correct_preds = np.sum(p == l)
        only_longs = np.sum(only_long == l)

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        total_hits += correct_preds
        total_data += outputs.size(0)
        total_ol += only_longs
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    #print(f"test accuracy: {accuracy*100:.2f}% | test only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, total_hits, total_data, total_ol

  def save_model(self, val_acc, test_acc, epoch, validation_year, test_year):
    PATH = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}/reset:{self.configs.reset_model}_valYear:{validation_year}_testYear:{test_year}.pt"
    torch.save({
            'model_state_dict': self.model.state_dict(),
            'model_optimizer_state_dict': self.model_optim.state_dict(),
            'val_acc': val_acc,
            'validation_year': validation_year,
            'test_acc': test_acc,
            'test_year': test_year,
            'configs': self.configs}, PATH)
    print(f"Model Saved at {PATH}")
    return PATH

  def load_model(self, path):
    checkpoint = torch.load(path, map_location=torch.device(self.device))
    self.model.load_state_dict(checkpoint['model_state_dict'])
    self.model_optim.load_state_dict(checkpoint['model_optimizer_state_dict'])
    print(f"Model loaded from {path}")
    print(f"Model Validation Accuracy: {checkpoint['val_acc']*100:.2f}% | Test Accuracy: {checkpoint['test_acc']*100:.2f}%")
    try:
      self.configs = checkpoint['configs']
    except:
      print("No Configs Found in torch file")
    return

## Training

### Simple DateTime

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    ##
    freq: str = 'd'
    task_name: str = 'classification'
    num_class: int = 2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 8
    factor: int = 1
    enc_in: int = 35
    dec_in: int = 35
    c_out: int = 1
    d_model: int = 512
    d_ff: int = 2048
    moving_avg: int = 25
    factor: int = 1
    distil: bool = True
    output_attention: bool = False
    patience: int = 3
    learning_rate: float = 0.0001
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.05
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = False

    ## Training
    run_name: str = "aapl_nopretrained_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "aapl" #"^SPX" #"goog amzn wmt xom brk-a lly ge lin pld aapl nee"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "autoformer_classification"


In [ ]:
configs = Args()
trainer = Trainer(configs)

Loading following tickers: ['aapl']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
Directory Already Exists at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[aapl]_valYrs:2_testYrs1_reset:False_aapl_nopretrained_run1


In [ ]:
trainer.run()

  0%|          | 0/21 [00:00<?, ?it/s]

Training Years: 1990~1999 | Training Accuracy: 50.95% | Training OL Accuracy: 47.15%
Year: 2000~2001 Validation Accuracy: 47.60% | Only Long Accuracy: 46.80% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 46.03% | Only Long Accuracy: 43.65%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[aapl]_valYrs:2_testYrs1_reset:False_aapl_nopretrained_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Training Years: 1990~1999 | Training Accuracy: 50.12% | Training OL Accuracy: 47.15%
Year: 2000~2001 Validation Accuracy: 50.60% | Only Long Accuracy: 46.80% | Highest Validation Accuracy: 47.60%
Year: 2002 | Test Accuracy: 49.21% | Only Long Accuracy: 43.65%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[aapl]_valYrs:2_testYrs1_reset:False_aapl_nopretrained_run1/reset:False_valYear:[2000, 2001

  5%|▍         | 1/21 [03:28<1:09:24, 208.21s/it]

Training Years: 1990~1999 | Training Accuracy: 49.84% | Training OL Accuracy: 47.15%
Year: 2000~2001 Validation Accuracy: 49.20% | Only Long Accuracy: 46.80% | Highest Validation Accuracy: 52.60%
Year: 2002 | Test Accuracy: 46.83% | Only Long Accuracy: 43.65%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 52.60% | Highest Test Accuracy in 2002: 46.83%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[aapl]_valYrs:2_testYrs1_reset:False_aapl_nopretrained_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Model Validation Accuracy: 52.60% | Test Accuracy: 48.02%


<ipython-input-14-c6f38c2cb730>:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


...

Training Years: 2009~2018 | Training Accuracy: 51.59% | Training OL Accuracy: 52.78%
Year: 2019~2020 Validation Accuracy: 56.04% | Only Long Accuracy: 56.24% | Highest Validation Accuracy: 58.22%
Year: 2021 | Test Accuracy: 49.60% | Only Long Accuracy: 53.17%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2019~2020: 58.22% | Highest Test Accuracy in 2021: 49.60%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[aapl]_valYrs:2_testYrs1_reset:False_aapl_nopretrained_run1/reset:False_valYear:[2019, 2020]_testYear:2021.pt
Model Validation Accuracy: 58.22% | Test Accuracy: 53.57%


<ipython-input-14-c6f38c2cb730>:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Training Years: 2010~2019 | Training Accuracy: 51.19% | Training OL Accuracy: 53.02%
Year: 2020~2021 Validation Accuracy: 53.66% | Only Long Accuracy: 53.66% | Highest Validation Accuracy: 0.00%
Year: 2022 | Test Accuracy: 47.01% | Only Long Accuracy: 47.01%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[aapl]_valYrs:2_testYrs1_reset:False_aapl_nopretrained_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Training Years: 2010~2019 | Training Accuracy: 51.83% | Training OL Accuracy: 53.02%
Year: 2020~2021 Validation Accuracy: 50.10% | Only Long Accuracy: 53.66% | Highest Validation Accuracy: 53.66%
Year: 2022 | Test Accuracy: 49.80% | Only Long Accuracy: 47.01%
------------------------------------------------
Training Years: 2010~2019 | Training Accuracy: 52.38% | Training OL Accuracy: 53.02%
Year: 2020~2021 Validation Accuracy: 52.87% | Only Long Accuracy: 53.66% | Highest Validation Acc

100%|██████████| 21/21 [1:14:49<00:00, 213.77s/it]

Training Years: 2010~2019 | Training Accuracy: 51.51% | Training OL Accuracy: 53.02%
Year: 2020~2021 Validation Accuracy: 51.49% | Only Long Accuracy: 53.66% | Highest Validation Accuracy: 55.45%
Year: 2022 | Test Accuracy: 47.81% | Only Long Accuracy: 47.01%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2020~2021: 55.45% | Highest Test Accuracy in 2022: 47.81%
***********************************************************
=============================New Training Set====================================

Validation Accuracy in 2000~2001 : 52.60% | Test Accuracy in 2002: 48.02%
Validation Accuracy in 2001~2002 : 55.80% | Test Accuracy in 2003: 46.43%
Validation Accuracy in 2002~2003 : 53.97% | Test Accuracy in 2004: 44.44%
Validation Accuracy in 2003~2004 : 49.01% | Test Accuracy in 2005: 44.84%
Validation Accuracy in 2004~2005 : 52.58% | Test Accuracy in 2006: 45.82%
Validation Accuracy in 2005~2

### OneHot DateTime

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    ##
    freq: str = 'd'
    task_name: str = 'classification'
    num_class: int = 2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 8
    factor: int = 1
    enc_in: int = 35
    dec_in: int = 35
    c_out: int = 1
    d_model: int = 512
    d_ff: int = 2048
    moving_avg: int = 25
    factor: int = 1
    distil: bool = True
    output_attention: bool = False
    patience: int = 3
    learning_rate: float = 0.0001
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.05
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    one_hot_datetime: bool = True
    datetime_features: int = 55

    ## Training
    run_name: str = "onehot_datetime_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "^SPX" #"^SPX" #"goog amzn wmt xom brk-a lly ge lin pld aapl nee"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "autoformer_classification"


In [ ]:
configs = Args()
trainer = Trainer(configs)

Loading following tickers: ['^SPX']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
DateTime is one-hot: True
Directory Already Exists at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[^SPX]_valYrs:2_testYrs1_reset:False_onehot_datetime_run1


In [ ]:
trainer.run()

  0%|          | 0/21 [00:00<?, ?it/s]

Training Years: 1990~1999 | Training Accuracy: 54.11% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 52.00% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 45.63% | Only Long Accuracy: 44.44%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[^SPX]_valYrs:2_testYrs1_reset:False_onehot_datetime_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Training Years: 1990~1999 | Training Accuracy: 63.13% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 53.60% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 52.00%
Year: 2002 | Test Accuracy: 46.83% | Only Long Accuracy: 44.44%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[^SPX]_valYrs:2_testYrs1_reset:False_onehot_datetime_run1/reset:False_valYear:[2000, 2001]_te

  5%|▍         | 1/21 [01:40<33:20, 100.03s/it]

Training Years: 1990~1999 | Training Accuracy: 100.00% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 76.40% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 76.80%
Year: 2002 | Test Accuracy: 69.05% | Only Long Accuracy: 44.44%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 76.80% | Highest Test Accuracy in 2002: 69.05%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[^SPX]_valYrs:2_testYrs1_reset:False_onehot_datetime_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Model Validation Accuracy: 76.80% | Test Accuracy: 67.06%


<ipython-input-24-09f888f4e133>:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


...

Training Years: 2009~2018 | Training Accuracy: 100.00% | Training OL Accuracy: 54.45%
Year: 2019~2020 Validation Accuracy: 86.34% | Only Long Accuracy: 58.42% | Highest Validation Accuracy: 87.13%
Year: 2021 | Test Accuracy: 84.52% | Only Long Accuracy: 56.75%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2019~2020: 87.13% | Highest Test Accuracy in 2021: 84.52%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[^SPX]_valYrs:2_testYrs1_reset:False_onehot_datetime_run1/reset:False_valYear:[2019, 2020]_testYear:2021.pt
Model Validation Accuracy: 87.13% | Test Accuracy: 86.11%


<ipython-input-24-09f888f4e133>:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Training Years: 2010~2019 | Training Accuracy: 98.25% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 81.58% | Only Long Accuracy: 57.03% | Highest Validation Accuracy: 0.00%
Year: 2022 | Test Accuracy: 85.26% | Only Long Accuracy: 43.03%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_classification/[^SPX]_valYrs:2_testYrs1_reset:False_onehot_datetime_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Training Years: 2010~2019 | Training Accuracy: 96.18% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 81.19% | Only Long Accuracy: 57.03% | Highest Validation Accuracy: 81.58%
Year: 2022 | Test Accuracy: 82.07% | Only Long Accuracy: 43.03%
------------------------------------------------
Training Years: 2010~2019 | Training Accuracy: 98.73% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 83.96% | Only Long Accuracy: 57.03% | Highest Validation Accur

100%|██████████| 21/21 [35:08<00:00, 100.42s/it]

Training Years: 2010~2019 | Training Accuracy: 100.00% | Training OL Accuracy: 54.85%
Year: 2020~2021 Validation Accuracy: 86.14% | Only Long Accuracy: 57.03% | Highest Validation Accuracy: 86.34%
Year: 2022 | Test Accuracy: 88.45% | Only Long Accuracy: 43.03%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2020~2021: 86.34% | Highest Test Accuracy in 2022: 88.45%
***********************************************************
=============================New Training Set====================================

Validation Accuracy in 2000~2001 : 76.80% | Test Accuracy in 2002: 67.06%
Validation Accuracy in 2001~2002 : 75.40% | Test Accuracy in 2003: 69.44%
Validation Accuracy in 2002~2003 : 74.40% | Test Accuracy in 2004: 81.75%
Validation Accuracy in 2003~2004 : 80.36% | Test Accuracy in 2005: 80.56%
Validation Accuracy in 2004~2005 : 85.12% | Test Accuracy in 2006: 79.68%
Validation Accuracy in 2005~

## Dow30 Pretrained

### Pretraining on Dow30

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    ##
    freq: str = 'd'
    task_name: str = 'classification'
    num_class: int = 2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 8
    factor: int = 1
    enc_in: int = 35
    dec_in: int = 35
    c_out: int = 1
    d_model: int = 512
    d_ff: int = 2048
    moving_avg: int = 25
    factor: int = 1
    distil: bool = True
    output_attention: bool = False
    patience: int = 3
    learning_rate: float = 0.0001
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.05
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023

    ## Training
    run_name: str = "dow30_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "amzn axp amgn aapl ba cat csco cvx gs hd hon ibm intc jnj ko jpm mcd mmm mrk msft nke pg trv unh crm vz v wmt dis"
    rolling_window: int = 7 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "autoformer_rolling"

In [ ]:
configs = Args()
trainer = Trainer(configs)

Loading following tickers: ['amzn', 'axp', 'amgn', 'aapl', 'ba', 'cat', 'csco', 'cvx', 'gs', 'hd', 'hon', 'ibm', 'intc', 'jnj', 'ko', 'jpm', 'mcd', 'mmm', 'mrk', 'msft', 'nke', 'pg', 'trv', 'unh', 'crm', 'vz', 'v', 'wmt', 'dis']

Dataset Start Year: 2010 | End Year: 2023
years: [2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022
 2023]
Directory Already Exists at /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_dow30_run1


In [ ]:
trainer.run()

  0%|          | 0/4 [00:00<?, ?it/s]

Training Years: 2010~2016 | Training Accuracy: 50.74% | Training OL Accuracy: 51.99%
Year: 2017~2018 Validation Accuracy: 48.74% | Only Long Accuracy: 53.41% | Highest Validation Accuracy: 0.00%
Year: 2019 | Test Accuracy: 48.82% | Only Long Accuracy: 55.35%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_dow30_run1/reset:False_valYear:[2017, 2018]_testYear:2019.pt
Training Years: 2010~2016 | Training Accuracy: 50.49% | Training OL Accuracy: 51.99%
Year: 2017~2018 Validation Accuracy: 48.08% | Only Long Accuracy: 53.41% | Highest Validation Accuracy: 48.74%
Year: 2019 | Test Accuracy: 47.70% | Only Long Accuracy: 55.35%
------------------------------------------------
Training Years: 2010~2016 | Training Accuracy: 50.54% | Training OL Accuracy: 51.99%
Year: 2

  0%|          | 0/4 [12:48<?, ?it/s]


KeyboardInterrupt: 

### FineTuning

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    ##
    freq: str = 'd'
    task_name: str = 'classification'
    num_class: int = 2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 8
    factor: int = 1
    enc_in: int = 35
    dec_in: int = 35
    c_out: int = 1
    d_model: int = 512
    d_ff: int = 2048
    moving_avg: int = 25
    factor: int = 1
    distil: bool = True
    output_attention: bool = False
    patience: int = 3
    learning_rate: float = 0.0001
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.05
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023

    ## Training
    run_name: str = "aapl_run1"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "aapl"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "autoformer_rolling"

In [ ]:
configs = Args()
trainer = Trainer(configs)
path = "/content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_dow30_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt"
trainer.load_model(path)

Loading following tickers: ['aapl']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
Save File Directory Made at /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1

Model loaded from /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[amzn_axp_amgn_aapl_ba_cat_csco_cvx_gs_hd_hon_ibm_intc_jnj_ko_jpm_mcd_mmm_mrk_msft_nke_pg_trv_unh_crm_vz_v_wmt_dis]_valYrs:2_testYrs1_reset:False_dow30_run1/reset:False_valYear:[2020, 2021]_testYear:2022.pt
Model Validation Accuracy: 51.96% | Test Accuracy: 49.42%


<ipython-input-15-c6f38c2cb730>:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


In [ ]:
trainer.run()

  0%|          | 0/21 [00:00<?, ?it/s]

Training Years: 1990~1999 | Training Accuracy: 47.94% | Training OL Accuracy: 47.15%
Year: 2000~2001 Validation Accuracy: 54.80% | Only Long Accuracy: 46.80% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 53.97% | Only Long Accuracy: 43.65%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Training Years: 1990~1999 | Training Accuracy: 51.38% | Training OL Accuracy: 47.15%
Year: 2000~2001 Validation Accuracy: 53.60% | Only Long Accuracy: 46.80% | Highest Validation Accuracy: 54.80%
Year: 2002 | Test Accuracy: 55.16% | Only Long Accuracy: 43.65%
------------------------------------------------
Training Years: 1990~1999 | Training Accuracy: 52.25% | Training OL Accuracy: 47.15%
Year: 2000~2001 Validation Accuracy: 53.20% | Only Long Accuracy: 46.80% | Highest Validation Accuracy: 54.80%
Year: 

  5%|▍         | 1/21 [03:42<1:14:10, 222.50s/it]

Training Years: 1990~1999 | Training Accuracy: 52.97% | Training OL Accuracy: 47.15%
Year: 2000~2001 Validation Accuracy: 53.60% | Only Long Accuracy: 46.80% | Highest Validation Accuracy: 54.80%
Year: 2002 | Test Accuracy: 57.54% | Only Long Accuracy: 43.65%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 54.80% | Highest Test Accuracy in 2002: 57.54%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Model Validation Accuracy: 54.80% | Test Accuracy: 53.97%


<ipython-input-15-c6f38c2cb730>:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


...

Training Years: 1994~2003 | Training Accuracy: 51.77% | Training OL Accuracy: 47.56%
Year: 2004~2005 Validation Accuracy: 44.84% | Only Long Accuracy: 57.54% | Highest Validation Accuracy: 0.00%
Year: 2006 | Test Accuracy: 51.79% | Only Long Accuracy: 45.82%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2004, 2005]_testYear:2006.pt
Training Years: 1994~2003 | Training Accuracy: 53.20% | Training OL Accuracy: 47.56%
Year: 2004~2005 Validation Accuracy: 44.64% | Only Long Accuracy: 57.54% | Highest Validation Accuracy: 44.84%
Year: 2006 | Test Accuracy: 52.19% | Only Long Accuracy: 45.82%
------------------------------------------------
Training Years: 1994~2003 | Training Accuracy: 49.46% | Training OL Accuracy: 47.56%
Year: 2004~2005 Validation Accuracy: 41.07% | Only Long Accuracy: 57.54% | Highest Validation Accuracy: 44.84%
Year: 

 24%|██▍       | 5/21 [18:34<59:29, 223.07s/it]  

Training Years: 1994~2003 | Training Accuracy: 51.05% | Training OL Accuracy: 47.56%
Year: 2004~2005 Validation Accuracy: 42.66% | Only Long Accuracy: 57.54% | Highest Validation Accuracy: 48.61%
Year: 2006 | Test Accuracy: 53.39% | Only Long Accuracy: 45.82%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2004~2005: 48.61% | Highest Test Accuracy in 2006: 53.39%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2004, 2005]_testYear:2006.pt
Model Validation Accuracy: 48.61% | Test Accuracy: 49.00%


<ipython-input-15-c6f38c2cb730>:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Training Years: 1995~2004 | Training Accuracy: 51.57% | Training OL Accuracy: 48.55%
Year: 2005~2006 Validation Accuracy: 50.30% | Only Long Accuracy: 52.29% | Highest Validation Accuracy: 0.00%
Year: 2007 | Test Accuracy: 51.79% | Only Long Accuracy: 56.18%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2005, 2006]_testYear:2007.pt
Training Years: 1995~2004 | Training Accuracy: 50.97% | Training OL Accuracy: 48.55%
Year: 2005~2006 Validation Accuracy: 51.09% | Only Long Accuracy: 52.29% | Highest Validation Accuracy: 50.30%
Year: 2007 | Test Accuracy: 49.40% | Only Long Accuracy: 56.18%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/autoformer_rolling/[aapl]_valYrs:2_testYrs1_reset:False_aapl_run1/reset:False_valYear:[2005, 2006]_testYear:2007.pt
Training Years: 1995~